# Testing Individual components of the FV HE scheme

In [1]:
import random

from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encryptor import Encryptor
from syft.frameworks.torch.he.fv.decryptor import Decryptor
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.modulus import SeqLevelType
from syft.frameworks.torch.he.fv.evaluator import Evaluator

## Keygeneration

In [2]:
poly_modulus = 64
bit_sizes= [40]
plain_modulus = 64
ctx = Context(EncryptionParams(poly_modulus, CoeffModulus().create(poly_modulus, bit_sizes), plain_modulus))
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [3]:
print(ctx.param.coeff_modulus)

AttributeError: 'Context' object has no attribute 'param'

In [4]:
# print(len(sk.data))
print('secret key values : ', sk.data)

secret key values :  [[1, 1, 0, 0, 1, 1099511623296, 1099511623296, 1099511623296, 1099511623296, 1099511623296, 0, 0, 1, 0, 0, 1099511623296, 1099511623296, 0, 1099511623296, 1, 1099511623296, 1, 1099511623296, 1099511623296, 1, 1099511623296, 1099511623296, 1, 0, 0, 1099511623296, 1, 1, 0, 1, 1099511623296, 0, 1099511623296, 0, 1, 1, 1, 1, 0, 0, 1099511623296, 0, 1099511623296, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1099511623296, 1099511623296, 1, 1, 1099511623296]]


In [5]:
# print(pk.data)
# print('public key values : ', pk.data)

## Integer Encoder
Encodes Integer values to Plaintext object

In [6]:
int_encoder = IntegerEncoder(ctx)
ri1 = random.randint(0,10)
ri2 = random.randint(0,10)
ri3 = random.randint(0,10)
pt1 = int_encoder.encode(ri1)
pt2 = int_encoder.encode(ri2)
pt3 = int_encoder.encode(ri3)
print(pt1.data,"   ", pt2.data, "   ", pt3.data)
# print('plaintext data',plaintext.data)

[0, 1, 1]     [1, 0, 1]     [1]


### Decodes back to Integer

In [7]:
print(int_encoder.decode(pt1))
print(int_encoder.decode(pt2))
print(int_encoder.decode(pt3))

6
5
1


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [8]:
encrypter = Encryptor(ctx, pk)

In [9]:
ct1 = encrypter.encrypt(pt1)
ct2 = encrypter.encrypt(pt2)
ct3 = encrypter.encrypt(pt3)

Encrypt Plaintext to ciphertext using secret_key

## Decryptor
Decrypt Ciphertext to Plaintext using secret_key

In [10]:
decrypter = Decryptor(ctx, sk)

In [11]:
dec1 = decrypter.decrypt(ct1)
dec2 = decrypter.decrypt(ct2)
dec3 = decrypter.decrypt(ct3)

In [12]:
print(int_encoder.decode(dec1), "   ", int_encoder.decode(dec2), "   ", int_encoder.decode(dec3))

6     5     1


## Evaluator

In [13]:
eval = Evaluator(ctx)

In [14]:
cc12 = eval.add(ct1, ct2)
cc12 = decrypter.decrypt(cc12)
print(int_encoder.decode(cc12))

11


In [15]:
pc12 = eval.add(pt1, ct2)
pc12 = decrypter.decrypt(pc12)
print(int_encoder.decode(pc12))

11


In [16]:
pp12 = eval.add(pt1, pt2)
print(int_encoder.decode(pp12))

11


### Verify result

In [17]:
assert int_encoder.decode(cc12) == int_encoder.decode(pc12) == int_encoder.decode(pp12) == ri1+ri2

In [18]:
result = eval._mul_cipher_cipher(ct1, ct2)
print("\n\nct1 :",ct1.data)
print("\n\nct2 :",ct2.data)
print('\n\n')

result = decrypter.decrypt(result)
result = int_encoder.decode(result)

print('final result: ', result)



ct1 : [[[1067512001505, 350015162115, 311406443496, 80496350393, 524373797622, 742644556718, 62936579817, 52310147176, 385997988137, 118689484330, 498592338647, 719074861900, 1003447586295, 858667985260, 966369592686, 367520268254, 1057660243718, 880915047276, 416861344029, 108098925421, 181357590484, 618737843735, 507024383305, 897795528943, 129496199136, 481259278936, 202137923983, 453766831890, 890176989942, 806284886737, 715515666539, 690636942381, 262248549485, 221766004483, 692635831460, 607105351283, 951468695372, 586734520164, 157844984804, 924728710547, 395194259874, 438000719231, 610057978406, 458477054482, 439488609397, 1087479101999, 1091650373606, 762559739334, 1248042238, 155190016541, 103883267053, 992901830940, 354167052202, 878581966717, 904072168396, 1035942765410, 240536204934, 490619440102, 813891009974, 962058061041, 357201344497, 105272954871, 246881996360, 999577688370]], [[503483181481, 574119001787, 746005074219, 108394207121, 81257705747, 426364969775, 92288

In [19]:
print(ri1 * ri2, "    ", result)
assert ri1 * ri2 == result

30      30


## Try Relinearization operation

In [46]:
poly_modulus = 64
bit_sizes = [40, 40]
plain_modulus = 64
ctx = Context(EncryptionParams(poly_modulus, CoeffModulus().create(poly_modulus, bit_sizes), plain_modulus))
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [47]:
relin_keys = keygenerator.get_relin_keys()
# relin_keys

In [48]:
int_encoder = IntegerEncoder(ctx)
a = int_encoder.encode(10)
b = int_encoder.encode(19)
c = int_encoder.encode(26)

In [49]:
encrypter = Encryptor(ctx, pk)
decrypter = Decryptor(ctx, sk)
eval = Evaluator(ctx)

In [59]:
relin_prod_ab = eval.relin(eval.mul(encrypter.encrypt(a), encrypter.encrypt(b)), relin_keys)
print(int_encoder.decode(decrypter.decrypt(relin_prod_ab)))

assert len(relin_prod_ab.data) == 2

190


In [62]:
relin_prod_abc = eval.relin(eval.mul(relin_prod_ab, encrypter.encrypt(c)), relin_keys)
print(int_encoder.decode(decrypter.decrypt(relin_prod_abc)))

assert len(relin_prod_abc.data) == 2

4940


In [63]:
final = eval.relin(eval.mul(relin_prod_ab, relin_prod_abc), relin_keys)
print(int_encoder.decode(decrypter.decrypt(final)))

assert len(final.data) == 2

938600
